In [1]:
import sys

import matplotlib.pyplot as plt
import pandas as pd
import pydiffvg

In [2]:
# %load_ext autoreload
# %autoreload 2
import covergan


In [3]:
import importlib, sys
# importlib.reload(covergan)

In [4]:
import pandas as pd
data = pd.read_csv('/home/dmitriy/OVE/official_data/artemis_dataset_release_v0.csv')

In [5]:
# %load_ext autoreload
# %autoreload 2
import covergan

In [6]:
from colorer.music_palette_dataset import PaletteDataset,ImageDataset
dataset = PaletteDataset(8,1000,'name','/home/dmitriy/OVE/covergan/weights','/home/dmitriy/OVE/wikiart',pandas_dir='/home/dmitriy/OVE/official_data/artemis_dataset_release_v0.csv')
dataset_image = ImageDataset(8,100,'name','/home/dmitriy/OVE/covergan/weights','/home/dmitriy/OVE/wikiart',pandas_dir='/home/dmitriy/OVE/official_data/artemis_dataset_release_v0.csv')

In [7]:
import torch

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
from colorer.models.colorer_dropout import Colorer2
z_dim=32
num_gen_layers=5
colors_count = 8
colorer = Colorer2(
        z_dim=z_dim,
        audio_embedding_dim=9,
        num_layers=num_gen_layers,
        colors_count=colors_count)

In [10]:
colorer

Colorer2(
  (model_): Sequential(
    (0): Linear(in_features=41, out_features=35, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=35, out_features=29, bias=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): BatchNorm1d(29, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Linear(in_features=29, out_features=23, bias=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): BatchNorm1d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2)
    (12): Linear(in_features=23, out_features=17, bias=True)
    (13): Dropout(p=0.2, inplace=False)
    (14): BatchNorm1d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): LeakyReLU(negative_slope=0.2)
    (16): Linear(in_features=17, out_features=24, bi

In [11]:
from colorer.models.gan_colorer import ColorerDiscriminator

In [12]:
disc = ColorerDiscriminator(audio_embedding_dim=9, num_layers=2,colors_count= colors_count).to(device)

In [13]:
disc

ColorerDiscriminator(
  (adv_layer): Sequential(
    (0): Linear(in_features=33, out_features=17, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=17, out_features=8, bias=True)
    (4): Sigmoid()
  )
)

In [14]:
def weighted_mse_loss(input, target, weight=None):
    if weight is None:
        max_weight = input.size()[1]
        weight = torch.tensor([(max_weight - i // 3) // 3 for i in range(max_weight)]).to(input.device)
        weight = weight.repeat((len(input), 1))
    return (weight * (input - target) ** 2).mean()

In [15]:
def get_noise(n_samples, input_dim, device):
    """
    Function for creating noise vectors: Given the dimensions (n_samples, input_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        input_dim: the dimension of the input vector, a scalar
        device: the device type
    """
    return torch.randn(n_samples, input_dim, device=device)

In [16]:
from torch.utils.data.dataloader import DataLoader
batch_size=32
train_dataloader = DataLoader(dataset,drop_last=True, batch_size=batch_size, shuffle=True)

In [17]:
def get_checkpoint_filename(checkpoint_root: str, checkpoint_name: str, epoch: int = None) -> str:
    suffix = f"-{epoch}" if epoch is not None else ""
    return f"{checkpoint_root}/{checkpoint_name}{suffix}.pt"

In [18]:
from captioner_train import logger
import os


def load_checkpoint(checkpoint_root: str, checkpoint_name: str,
                    models) -> int:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    filename = get_checkpoint_filename(checkpoint_root, checkpoint_name)

    if os.path.isfile(filename):
        logger.info(f"Found {filename}, loading")
        checkpoint = torch.load(filename, map_location=device)
        for i, model in enumerate(models):
            model.load_state_dict(checkpoint[f"{i}_state_dict"])
            print('Loaded')
        epochs_done = checkpoint[f"epochs_done"]
        logger.info(f"{filename} loaded")
        return epochs_done
    else:
        return 0

In [19]:
def save_checkpoint(checkpoint_root: str, checkpoint_name: str, epochs_done: int, backup_epochs: int,
                    models):
    checkpoint_dict = {}
    for i, model in enumerate(models):
        checkpoint_dict[f"{i}_state_dict"] = model.state_dict()
    checkpoint_dict[f"epochs_done"] = epochs_done

    if not backup_epochs:
        # Unconditional save
        filename = get_checkpoint_filename(checkpoint_root, checkpoint_name)
        torch.save(checkpoint_dict, filename)
        logger.info(f"{filename} saved")
    if backup_epochs and epochs_done and epochs_done % backup_epochs == 0:
        # Regular backup
        filename = get_checkpoint_filename(checkpoint_root, checkpoint_name, epochs_done)
        torch.save(checkpoint_dict, filename)
        logger.info(f"Backup {filename} saved")

In [20]:
disc

ColorerDiscriminator(
  (adv_layer): Sequential(
    (0): Linear(in_features=33, out_features=17, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=17, out_features=8, bias=True)
    (4): Sigmoid()
  )
)

In [21]:
z_dim=9
num_gen_layers=5
colors_count = 8
colorer = Colorer2(
        z_dim=z_dim,
        audio_embedding_dim=9,
        num_layers=num_gen_layers,
        colors_count=colors_count)

In [22]:
from tqdm.notebook import tqdm
import logging
import torch
from torch.utils.data.dataloader import DataLoader
n_epochs = 5# training_params["n_epochs"]
lr = 3e-4
z_dim = 9
checkpoint_root ='/home/dmitriy/OVE/covergan/weights'
backup_epochs = 1
disc_slices = 6
colors_count = 8
gen_opt = torch.optim.Adam(colorer.parameters(), lr=lr)
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
criterion = torch.nn.MSELoss()

# model_name = f'colorer_{colorer.color_type}_{colorer.colors_count}_colors'
model_name = f'colorer_{colorer.colors_count}_colors_{train_dataloader.dataset.sorted_color}'
print("Trying to load checkpoint.")
epochs_done = load_checkpoint(checkpoint_root, model_name, [colorer, gen_opt])

if epochs_done:
    logger.info(f"Loaded a checkpoint with {epochs_done} epochs done")

log_interval = 5
# for epoch in range(epochs_done + 1, n_epochs + epochs_done + 1):
#     colorer.train()
#     running_D_test_loss = 0.0
#     running_G_test_loss = 0.0
#     count = 0
#     for X_batch, y_batch in tqdm(train_dataloader):
#         torch.cuda.empty_cache()
#         y_batch = y_batch.to(device)
#
#         cur_batch_size = len(X_batch)
#         X_batch = X_batch.float().to(device)
#         X_batch_disc = X_batch
#
#         real_outputs = disc(X_batch_disc, y_batch)
#         real_label = torch.ones(y_batch.shape[0], colors_count).to(device)
#         z = get_noise(cur_batch_size, z_dim, device=device)
#
#         fake_inputs = colorer(z, X_batch_disc)
#         fake_outputs = disc(X_batch_disc, fake_inputs)
#
#         fake_label = torch.zeros(fake_inputs.shape[0], colors_count).to(device)
#         outputs = torch.cat((real_outputs, fake_outputs), dim=0)
#         targets = torch.cat((real_label, fake_label), dim=0)
#         D_loss = criterion(outputs, targets)
#         running_D_test_loss += D_loss.item()
#         disc_opt.zero_grad()
#         D_loss.backward()
#         disc_opt.step()
#         gen_opt.zero_grad()
#         z = get_noise(cur_batch_size, z_dim, device=device)
#
#         fake_inputs = colorer(z, X_batch_disc)
#         fake_outputs = disc(X_batch_disc, fake_inputs)
#         fake_targets = torch.ones(fake_inputs.shape[0], colors_count).to(device)
#         G_loss = criterion(fake_outputs, fake_targets)
#         gen_opt.zero_grad()
#         G_loss.backward()
#         gen_opt.step()
#         running_G_test_loss += G_loss.item()
#
#         if (count + 1) % log_interval == 0:
#                 print('Epoch {} Iteration {}: discriminator_loss {:.3f} generator_loss {:.3f}'
#                       .format(epoch, count, D_loss.item(), G_loss.item()))
#                 save_checkpoint(checkpoint_root, model_name, epoch, backup_epochs, [colorer, disc, gen_opt, disc_opt])
#         count+=1
#     avg_G_test_loss = running_G_test_loss / (count + 1)
#     avg_D_test_loss = running_D_test_loss / (count+ 1)
#     print('Train LOSS: colorer {}, disc {}'.format(avg_G_test_loss, avg_D_test_loss))


Trying to load checkpoint.


In [23]:
from outer.models.discriminator import Discriminator
from outer.models.my_gen_fixed_6figs32_good import MyGeneratorFixedSixFigs32Good

generator_type = MyGeneratorFixedSixFigs32Good
discriminator_type = Discriminator

In [24]:
num_gen_layers = 4
num_disc_conv_layers = 3
num_disc_linear_layers = 2
z_dim = 32  # Dimension of the noise vector
# z_dim = 512  # Dimension of the noise vector

# Painter properties
path_count = 3
path_segment_count = 4
disc_slices = 6
max_stroke_width = 0.01  # relative to the canvas size

In [25]:
gen = MyGeneratorFixedSixFigs32Good(9,9,5,400,max_stroke_width)

In [26]:
disc = Discriminator(400,9,3,2)

In [27]:
import matplotlib.pyplot as plt
dataset_image[6][1].shape[1]

400

In [28]:
def plot_losses(epoch: int, cur_step: int, display_steps: int, bin_steps: int, losses: [(str, [float])]):
    if cur_step % display_steps == 0 and cur_step > 0:
        loss_stats = []

        for loss_name, loss_values in losses:
            loss_mean = sum(loss_values[-display_steps:]) / display_steps
            if "loss" not in loss_name.lower() and "metric" not in loss_name.lower():
                loss_name += " Loss"
            loss_stats.append(f"{loss_name}: {loss_mean}")

            num_examples = (len(loss_values) // bin_steps) * bin_steps
            plt.plot(
                range(num_examples // bin_steps),
                torch.Tensor(loss_values[:num_examples]).view(-1, bin_steps).mean(1),
                label=loss_name
            )

        logger.info(f"Epoch {epoch} (step {cur_step}): " + ", ".join(loss_stats))
        plt.legend()
        print("Saving losses to png file")
        import covergan_train
        plt.savefig(f"{covergan_train.logger.plots_dir}/losses-{epoch}-{cur_step}.png")
        plt.show()
        plt.close()
    elif cur_step == 0:
        logger.info("The training is working")

In [29]:
from matplotlib.lines import Line2D
from torchvision.transforms.functional import to_pil_image
import numpy as np

In [30]:
def get_gradient_penalty(disc, real, fake, audio_embedding):
    # Mix the images together
    epsilon = torch.rand((real.size(0), 1, 1, 1), device=real.device)
    mixed_images = (real * epsilon + fake * (1 - epsilon)).requires_grad_(True)

    # Calculate the critic's scores on the mixed images
    mixed_scores = disc(mixed_images, audio_embedding)

    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(
        inputs=mixed_images,
        outputs=mixed_scores,
        grad_outputs=torch.ones_like(mixed_scores),
        create_graph=True,
        retain_graph=True,
    )[0]
    gradient = gradient.view(len(gradient), -1)

    # Calculate the magnitude of every row
    gradient_norm = gradient.norm(2, dim=1)

    # Penalize the mean squared distance of the gradient norms from 1
    penalty = ((gradient_norm - 1) ** 2).mean()
    return penalty

In [31]:
def plot_grad_flow(named_parameters, model_name: str, epoch=None, cur_step=None):
    """Plots the gradients flowing through different layers in the network during training.
    Can be used for checking for possible gradient vanishing/exploding problems.

    Usage: Plug this function in Trainer class after loss.backwards() as
    `plot_grad_flow(self.model.named_parameters())` to visualize the gradient flow
    """

    # Calculate the stats
    avg_grads = []
    max_grads = []
    layers = []
    for n, p in named_parameters:
        if p.requires_grad and ("bias" not in n) and p.grad is not None:
            layers.append(n)
            avg_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())

    # Initialize plot canvas
    fig = plt.figure()
    fig.set_size_inches(6, 6)

    # Plot
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.2, lw=1, color="c")  # Max gradients
    plt.bar(np.arange(len(max_grads)), avg_grads, alpha=0.2, lw=1, color="b")  # Mean gradients
    plt.hlines(0, 0, len(avg_grads) + 1, lw=2, color="k")  # Zero gradient line
    plt.xticks(range(0, len(avg_grads), 1), layers, rotation="vertical")

    # Set display options
    plt.xlim(left=0, right=len(avg_grads))
    plt.ylim(bottom=-0.001, top=0.02)  # Zoom in on the lower gradient regions
    plt.xlabel('Layers')
    plt.ylabel('average gradient')
    plt.title(f'Gradient flow in {model_name}')
    plt.grid(True)
    plt.legend(
        [
            Line2D([0], [0], color="c", lw=4),
            Line2D([0], [0], color="b", lw=4),
            Line2D([0], [0], color="k", lw=4)
        ],
        [
            'max-gradient',
            'mean-gradient',
            'zero-gradient'
        ]
    )

    fig.tight_layout()
    plt.plot()
    if epoch is not None:
        print("Saving grad flow to png file")
        import covergan_train
        plt.savefig(f"{covergan_train.logger.plots_dir}/grad-flow-{epoch}-{cur_step}.png")
    plt.show()
    plt.close()


In [34]:
def plot_real_fake_covers(real_cover_tensor: torch.Tensor, fake_cover_tensor: torch.Tensor,
                          disc_real_pred: torch.Tensor = None, disc_fake_pred: torch.Tensor = None,
                          epoch=None, cur_step=None, plot_saving_dir="./plots"):
    sample_count = 5  # max covers to draw

    real_cover_tensor = real_cover_tensor[:sample_count]
    fake_cover_tensor = fake_cover_tensor[:sample_count]

    rows = min(sample_count, len(real_cover_tensor))
    cols = 2
    fig = plt.figure()
    fig.set_size_inches(6 * cols, 6 * rows)
    for i in range(rows):
        real = real_cover_tensor[i]
        fake = fake_cover_tensor[i]

        real_pil = to_pil_image(real)
        fake_pil = to_pil_image(fake)

        real_score = disc_real_pred[i].item() if disc_real_pred is not None else None
        fake_score = disc_fake_pred[i].item() if disc_fake_pred is not None else None

        for (j, (pil, score)) in enumerate([(real_pil, real_score), (fake_pil, fake_score)]):
            plt.subplot(rows, cols, i * cols + j + 1)
            plt.imshow(pil)
            if score is not None:
                plt.text(10, 10, f'{score:.3f}', backgroundcolor='w', fontsize=40.0)
            plt.xticks([])
            plt.yticks([])

    fig.tight_layout()
    plt.plot()
    if epoch is not None:
        print("Saving covers to png file")
        import covergan_train
        # print(covergan_train.logger.plots_dir)
        plt.savefig(f"{covergan_train.logger.plots_dir}/covers-{epoch}-{cur_step}.png")
    plt.show()
    plt.close()

In [ ]:
palette_generator = colorer
def generate(z, audio_embedding_disc):
    if palette_generator is None:
        return gen(z, audio_embedding_disc)
    return gen(z, audio_embedding_disc, palette_generator=palette_generator)

logger.info(f'PyDiffVG uses GPU: {pydiffvg.get_use_gpu()}')
logger.info(gen)
logger.info(disc)

n_epochs = 1
disc_repeats = 1#Discriminator runs per iteration


disc_lr = gen_lr = 3e-4

z_dim = 9
disc_slices = 1
checkpoint_root = '/home/dmitriy/OVE/covergan/weights'
display_steps = 30
backup_epochs = 5
bin_steps = 20
plot_grad = False
c_lambda = 1

batch_size=8
dataloader = DataLoader(dataset_image,drop_last=True, batch_size=batch_size, shuffle=True)

gen_opt = torch.optim.Adam(gen.parameters(), lr=gen_lr, betas=(0.5, 0.9))
disc_opt = torch.optim.Adam(disc.parameters(), lr=disc_lr, betas=(0.5, 0.9))
cgan_out_name='cgan_out'
print("Trying to load checkpoint.")
epochs_done = load_checkpoint(checkpoint_root, cgan_out_name, [gen, disc, gen_opt, disc_opt])
if epochs_done:
    logger.info(f"Loaded a checkpoint with {epochs_done} epochs done")

cur_step = 0
generator_losses = []
discriminator_losses = []
shuffle_losses = []
val_metrics = []

disc_repeat_cnt = 0
mean_iteration_disc_loss, mean_shuffle_disc_loss = 0, 0
for epoch in range(epochs_done + 1, n_epochs + epochs_done + 1):
    for emotion,image in tqdm(dataloader):
        torch.cuda.empty_cache()
        emotion = emotion.float().to(device)
        print(emotion.shape)
        image = image.to(device)
        emotion_disc = emotion
        cur_batch_size = len(emotion)

        def get_fake_pred(should_detach: bool):
            # Get noise corresponding to the current batch_size
            z = get_noise(cur_batch_size, z_dim, device=device)

            if should_detach:
                with torch.no_grad():
                    fake_covers = generate(z, emotion_disc)
                    fig_params = None
            else:
                # fake_covers, fig_params = gen(z, audio_embedding_disc, emotions, return_figure_params=True)
                fake_covers = generate(z, emotion_disc)
                fig_params = None
            # Make sure that enough shapes were generated
            assert len(fake_covers) == len(image)
            fake_pred = disc(fake_covers, emotion_disc)

            assert len(fake_pred) == len(fake_covers)
            return fake_covers, fake_pred, fig_params

        # ### Update discriminator with fakes ###
        # Zero out the discriminator gradients
        disc_opt.zero_grad()

        fake_cover_tensor, disc_fake_pred, fig_params = get_fake_pred(should_detach=True)
        print(image.shape)
        disc_real_pred = disc(image.permute(0,3,1,2), emotion)

        # Make sure that enough predictions were made
        assert len(disc_real_pred) == len(image)
        # Shapes must match
        assert tuple(disc_fake_pred.shape) == tuple(disc_real_pred.shape)

        # gp = get_gradient_penalty(disc, image.permute(0,3,1,2).data, fake_cover_tensor.data,
        #                           emotion)
        print('HERE0')
        disc_loss = disc_fake_pred.mean() - disc_real_pred.mean()
        # Keep track of the average critic loss in this batch
        mean_iteration_disc_loss += disc_loss.item() / disc_repeats
        disc_repeat_cnt += 1
        print('HERE1')
        # Update gradients
        print(disc_loss)
        disc_loss.backward()
        if plot_grad:
            plot_grad_flow(disc.named_parameters(), "discriminator (fakes)", epoch=epoch, cur_step=cur_step)
        # Update optimizer
        disc_opt.step()

        # if USE_SHUFFLING:
        #     # ### Update discriminator with matching and shuffled cover-embedding pairs ###
        #     disc_opt.zero_grad()
        #
        #     disc_real_pred = disc(real_cover_tensor, audio_embedding_disc, emotions)
        #     disc_shuffle_pred = disc(shuffle_cover_tensor, audio_embedding_disc, emotions)
        #     disc_loss = disc_shuffle_pred.mean() - disc_real_pred.mean()
        #
        #     mean_shuffle_disc_loss += disc_loss.item() / disc_repeats
        #
        #     # Update gradients
        #     disc_loss.backward()
        #     if plot_grad:
        #         plot_grad_flow(disc.named_parameters(), "discriminator (shuffled pairs)", epoch=epoch,
        #                        cur_step=cur_step)
        #     disc_opt.step()
        print('HERE2')
        if disc_repeat_cnt == disc_repeats:
            # Keep track of the average discriminator loss
            discriminator_losses.append(mean_iteration_disc_loss)
            shuffle_losses.append(mean_shuffle_disc_loss)
            disc_repeat_cnt = 0
            mean_iteration_disc_loss, mean_shuffle_disc_loss = 0, 0

            # ### Update generator ###
            # Zero out the generator gradients
            gen_opt.zero_grad()

            # Getting fake shapes, same as in the loop above
            fake_cover_tensor, disc_fake_pred, fig_params = get_fake_pred(should_detach=False)

            # p_dist = torch.nn.PairwiseDistance()
            # dist_loss_sum = torch.tensor(0.0).to(fake_cover_tensor.device)
            # for figs in fig_params:
            #     cur_batch_dist_loss = torch.tensor(0.0).to(fake_cover_tensor.device)
            #     for ind_x, x in enumerate(figs):
            #         for ind_y in range(ind_x + 1, len(figs)):
            #             a = x["center_point"]
            #             b = figs[ind_y]["center_point"]
            #             cur_batch_dist_loss += 50000 / (p_dist(a, b)) ** 3
            #     dist_loss_sum += cur_batch_dist_loss / len(figs)
            #
            # dist_loss_sum = dist_loss_sum / len(fig_params)
            # dist_loss_sum.backward()
            # gen_loss = -disc_fake_pred.mean() + dist_loss_sum
            gen_loss = -disc_fake_pred.mean()
            gen_loss.backward()
            if plot_grad:
                plot_grad_flow(gen.named_parameters(), "generator", epoch=epoch, cur_step=cur_step)

            # Update the weights
            gen_opt.step()

            # Keep track of the generator losses
            generator_losses.append(gen_loss.item())

        plot_losses(epoch, cur_step, display_steps, bin_steps, [
            ("Generator", generator_losses),
            ("Discriminator Adversarial", discriminator_losses),
            ("Discriminator Mismatches", shuffle_losses)
        ])
        if cur_step % display_steps == 0:
            plot_real_fake_covers(image.permute(0,3,1,2), fake_cover_tensor, disc_real_pred, disc_fake_pred, epoch=epoch,
                                  cur_step=cur_step)
            save_checkpoint(checkpoint_root, cgan_out_name, epoch, 0, [gen, disc, gen_opt, disc_opt])
        cur_step += 1

PyDiffVG uses GPU: False
MyGeneratorFixedSixFigs32Good(
  (model_): Sequential(
    (0): Linear(in_features=18, out_features=223, bias=True)
    (1): BatchNorm1d(223, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Linear(in_features=223, out_features=428, bias=True)
    (4): BatchNorm1d(428, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=428, out_features=633, bias=True)
    (7): BatchNorm1d(633, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ELU(alpha=1.0)
    (9): Linear(in_features=633, out_features=838, bias=True)
    (10): BatchNorm1d(838, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ELU(alpha=1.0)
    (12): Linear(in_features=838, out_features=1040, bias=True)
    (13): Tanh()
  )
)
Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 24, kernel_size=(6, 6), stride=(4, 4), padding=(1, 1), bias=False)
    

Trying to load checkpoint.


  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([8, 9])
torch.Size([8, 400, 400, 3])
HERE0
HERE1
tensor(0.0771, grad_fn=<SubBackward0>)


In [ ]:
a=1